# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Check for GPU

import tensorflow as tf
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

if not tf.test.gpu_device_name():
    warnings.warn('No GPU found.')
else:
    print('Default GPU Device {}'.format(tf.test.gpu_device_name()))

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2949096893726493688
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 11324823962
locality {
  bus_id: 1
}
incarnation: 15893714868511435467
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0"
]
Default GPU Device /gpu:0


In [3]:
import helper

# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

loading dataset
loading dataset
Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [4]:
for sample_i in range(20):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))
    print('--')

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
--
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .
--
small_vocab_en Line 3:  california is usually quiet during march , and it is usually hot in june .
small_vocab_fr Line 3:  california est généralement calme en mars , et il est généralement chaud en juin .
--
small_vocab_en Line 4:  the united states is sometimes mild during june , and it is cold in september .
small_vocab_fr Line 4:  les états-unis est parfois légère en juin , et il fait froid en septembre .
--
small_vocab_en Line 5:  your least liked fruit is the grape , but my least liked is the apple .
small_vocab_fr Line 5:  votre moins aimé fruit est l

From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [5]:
import collections


english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [6]:
import project_tests as tests
from keras.preprocessing.text import Tokenizer

def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'lexicography': 14, 'lazy': 4, 'over': 5, 'prize': 6, 'the': 2, 'dog': 21, 'brown': 8, 'short': 9, 'jove': 11, 'of': 12, 'study': 13, 'a': 1, 'jumps': 15, 'sentence': 7, 'this': 17, 'by': 20, 'my': 18, 'quick': 3, 'fox': 19, 'won': 10, 'is': 16}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [2, 3, 8, 19, 15, 5, 2, 4, 21]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [20, 11, 18, 3, 13, 12, 14, 10, 1, 6]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [17, 16, 1, 9, 7]


Using TensorFlow backend.


In [7]:
import keras

In [8]:
text_sentences

['The quick brown fox jumps over the lazy dog .',
 'By Jove , my quick study of lexicography won a prize .',
 'This is a short sentence .']

### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [9]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences

def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    return pad_sequences(x, maxlen=length, padding='post')
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [ 2  3  8 19 15  5  2  4 21]
  Output: [ 2  3  8 19 15  5  2  4 21  0]
Sequence 2 in x
  Input:  [20 11 18  3 13 12 14 10  1  6]
  Output: [20 11 18  3 13 12 14 10  1  6]
Sequence 3 in x
  Input:  [17 16  1  9  7]
  Output: [17 16  1  9  7  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [10]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)

print('Data Preprocessed')

Data Preprocessed


In [11]:
np.array(preproc_english_sentences).shape

(137861, 15)

In [12]:
np.array(em.predict(tmp_x[:1])).shape

NameError: name 'em' is not defined

## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [ ]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [ ]:
from keras.layers import GRU, Input, Dense, TimeDistributed
from keras.models import Model, Sequential
from keras.layers import Activation, SimpleRNN
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """

    in_layer = Input(shape=input_shape[1:])
    gru_layer = GRU(output_sequence_length, return_sequences=True)(in_layer)
    out_layer = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(gru_layer)
    model = Model(inputs=in_layer, outputs=out_layer)

    learning_rate = 0.1
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

In [199]:
# Train model and print prediction

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

print("input shape:" + str(tmp_x.shape))
print("output sequence length: " + str(preproc_french_sentences.shape[1]))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index) + 1,
    len(french_tokenizer.word_index) + 1)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

input shape:(137861, 21, 1)
output sequence length: 21
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 7s - loss: 2.3700 - acc: 0.4783 - val_loss: 2.0182 - val_acc: 0.5067
Epoch 2/10
110288/110288 [==============================] - 6s - loss: 1.9274 - acc: 0.5274 - val_loss: 1.8695 - val_acc: 0.5401
Epoch 3/10
110288/110288 [==============================] - 6s - loss: 1.8266 - acc: 0.5479 - val_loss: 1.7908 - val_acc: 0.5538
Epoch 4/10
110288/110288 [==============================] - 6s - loss: 1.7722 - acc: 0.5567 - val_loss: 1.7487 - val_acc: 0.5577
Epoch 5/10
110288/110288 [==============================] - 6s - loss: 1.7293 - acc: 0.5608 - val_loss: 1.7138 - val_acc: 0.5703
Epoch 6/10
110288/110288 [==============================] - 6s - loss: 1.7224 - acc: 0.5646 - val_loss: 1.7156 - val_acc: 0.5631
Epoch 7/10
110288/110288 [==============================] - 6s - loss: 1.7050 - acc: 0.5689 - val_loss: 1.7045 - val_acc

In [115]:
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

srm_test = simple_model(tmp_x.shape, preproc_french_sentences.shape[1], len(english_tokenizer.word_index) + 1, len(french_tokenizer.word_index) + 1)
srm_test.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_40 (InputLayer)        (None, 21, 1)             0         
_________________________________________________________________
gru_39 (GRU)                 (None, 21, 21)            1449      
_________________________________________________________________
time_distributed_22 (TimeDis (None, 21, 345)           7590      
Total params: 9,039.0
Trainable params: 9,039.0
Non-trainable params: 0.0
_________________________________________________________________


In [13]:
from keras.layers import Activation, SimpleRNN

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [14]:
def indices_to_text(indices, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    
    return ' '.join([index_to_words[prediction] for prediction in indices])


In [16]:
from keras.layers.embeddings import Embedding

def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    embedding_size = 8
    learning_rate = 0.1
    
    in_layer = Input(shape=input_shape[1:])
    embedding_layer = Embedding(english_vocab_size, embedding_size)(in_layer)
    gru_layer = GRU(output_sequence_length, return_sequences=True)(embedding_layer)
    out_layer = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(gru_layer)
    model = Model(inputs=in_layer, outputs=out_layer)

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_embed_model(embed_model)

NameError: name 'Input' is not defined

In [15]:
indices_to_text(np.argmax(em.predict(tmp_x[:1])[0], 1), french_tokenizer)

NameError: name 'em' is not defined

In [219]:
# Reshaping the input to work with a embedding RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])

print("input shape:" + str(tmp_x.shape[1:]))
print("output sequence length: " + str(preproc_french_sentences.shape[1]))

# Train the neural network
em = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index) + 1,
    len(french_tokenizer.word_index) + 1)
em.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(em.predict(tmp_x[:1])[0], french_tokenizer))

input shape:(21,)
output sequence length: 21
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 7s - loss: 1.6792 - acc: 0.6166 - val_loss: 0.9337 - val_acc: 0.7398
Epoch 2/10
110288/110288 [==============================] - 6s - loss: 0.8002 - acc: 0.7685 - val_loss: 0.7146 - val_acc: 0.7882
Epoch 3/10
110288/110288 [==============================] - 6s - loss: 0.6841 - acc: 0.7951 - val_loss: 0.6508 - val_acc: 0.8033
Epoch 4/10
110288/110288 [==============================] - 6s - loss: 0.6432 - acc: 0.8046 - val_loss: 0.6342 - val_acc: 0.8066
Epoch 5/10
110288/110288 [==============================] - 6s - loss: 0.6133 - acc: 0.8128 - val_loss: 0.6161 - val_acc: 0.8113
Epoch 6/10
110288/110288 [==============================] - 6s - loss: 0.6003 - acc: 0.8165 - val_loss: 0.5949 - val_acc: 0.8189
Epoch 7/10
110288/110288 [==============================] - 6s - loss: 0.5898 - acc: 0.8192 - val_loss: 0.5731 - val_acc: 0.8258
E

I don't know why using the original length of the tokenizers yields bad results:

In [58]:
em_test = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index),
    len(french_tokenizer.word_index))
em_test.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=5, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(em_test.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/5
110288/110288 [==============================] - 7s - loss: nan - acc: 0.6457 - val_loss: 5.8406 - val_acc: 0.4093
Epoch 2/5
110288/110288 [==============================] - 6s - loss: nan - acc: 0.4082 - val_loss: 5.8406 - val_acc: 0.4093
Epoch 3/5
110288/110288 [==============================] - 6s - loss: nan - acc: 0.4082 - val_loss: 5.8406 - val_acc: 0.4093
Epoch 4/5
110288/110288 [==============================] - 6s - loss: nan - acc: 0.4082 - val_loss: 5.8406 - val_acc: 0.4093
Epoch 5/5
110288/110288 [==============================] - 6s - loss: nan - acc: 0.4082 - val_loss: 5.8406 - val_acc: 0.4093
<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [120]:
# summarizing the model to see its layer shapes

tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

embed_test = embed_model(tmp_x.shape, preproc_french_sentences.shape[1], len(english_tokenizer.word_index) + 1, len(french_tokenizer.word_index) + 1)
embed_test.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_44 (InputLayer)        (None, 21)                0         
_________________________________________________________________
embedding_31 (Embedding)     (None, 21, 21)            4200      
_________________________________________________________________
gru_43 (GRU)                 (None, 21, 21)            2709      
_________________________________________________________________
time_distributed_25 (TimeDis (None, 21, 345)           7590      
Total params: 14,499.0
Trainable params: 14,499.0
Non-trainable params: 0.0
_________________________________________________________________


In [179]:
def print_model_shape(model):
    for layer in model.layers:
        print("[" + layer.name + "] " + str(layer.input_shape) + " => " + str(layer.output_shape))

In [217]:
print_model_shape(em)

[input_84] (None, 21) => (None, 21)
[embedding_58] (None, 21) => (None, 21, 256)
[gru_79] (None, 21, 256) => (None, 21, 21)
[time_distributed_59] (None, 21, 21) => (None, 21, 345)


In [181]:
print_model_shape(srm_test)

[input_41] (None, 21, 1) => (None, 21, 1)
[gru_40] (None, 21, 1) => (None, 21, 21)
[time_distributed_23] (None, 21, 21) => (None, 21, 345)


In [145]:
np.array(range(100)).reshape((5, -1, 5)).reshape((-1))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [156]:
print(preproc_french_sentences.shape)
print(preproc_english_sentences.shape)

(137861, 21, 1)
(137861, 15)


In [154]:
preproc_french_sentences.shape
#preproc_english_sentences.shape

tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
print(tmp_x.shape)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
print(tmp_x.shape)




(137861, 21)
(137861, 21, 1)


In [166]:
print(np.array(range(3)).shape)
print(np.array(range(3)).reshape((-1, 1)).shape)

(3,)
(3, 1)


### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [194]:
from keras.layers import Bidirectional

# example: https://keras.io/layers/wrappers/ (search 'bidirectional')

def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    learning_rate = 0.1

    in_layer = Input(shape=input_shape[1:])
    gru_layer = Bidirectional(GRU(output_sequence_length, return_sequences=True))(in_layer)
    out_layer = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(gru_layer)
    model = Model(inputs=in_layer, outputs=out_layer)

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_bd_model(bd_model)

In [195]:
# Train and print predictions

# Reshaping the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bdm = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index) + 1,
    len(french_tokenizer.word_index) + 1)
bdm.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bdm.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 10s - loss: 1.8533 - acc: 0.5665 - val_loss: 1.4627 - val_acc: 0.6157
Epoch 2/10
110288/110288 [==============================] - 9s - loss: 1.4210 - acc: 0.6244 - val_loss: 1.3923 - val_acc: 0.6310
Epoch 3/10
110288/110288 [==============================] - 9s - loss: 1.3601 - acc: 0.6334 - val_loss: 1.3141 - val_acc: 0.6403
Epoch 4/10
110288/110288 [==============================] - 9s - loss: 1.2969 - acc: 0.6408 - val_loss: 1.2651 - val_acc: 0.6450
Epoch 5/10
110288/110288 [==============================] - 9s - loss: 1.7956 - acc: 0.5971 - val_loss: 1.5580 - val_acc: 0.6114
Epoch 6/10
110288/110288 [==============================] - 9s - loss: 1.4798 - acc: 0.6237 - val_loss: 1.4412 - val_acc: 0.6265
Epoch 7/10
110288/110288 [==============================] - 9s - loss: 1.4276 - acc: 0.6299 - val_loss: 1.4083 - val_acc: 0.6332
Epoch 8/10
110288/110288 [===================

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [239]:
from keras.layers import RepeatVector

def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    
    timesteps = output_sequence_length
    input_dim = input_shape[2]
    latent_dim = 8

    print('input shape:' + str(input_shape))
    print('timesteps: ' + str(timesteps))
    print('input dim: ' + str(input_dim))
    print('latent dim: ' + str(latent_dim))
    
    inputs = Input(shape=input_shape[1:])
    encoded = GRU(output_sequence_length)(inputs)

#    decoded = RepeatVector(timesteps)(encoded)
#    decoded = GRU(input_dim, return_sequences=True)(decoded)

#    sequence_autoencoder = Model(inputs, decoded)
#    encoder = Model(inputs, encoded)
    
    return encoder
tests.test_encdec_model(encdec_model)


# OPTIONAL: Train and Print prediction(s)

input shape:(137861, 15, 1)
timesteps: 21
input dim: 1
latent dim: 8


NameError: name 'encoder' is not defined

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [ ]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    return None
tests.test_model_final(model_final)


print('Final Model Loaded')

## Prediction (IMPLEMENTATION)

In [ ]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = None

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.argmax(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

## Submission
When you are ready to submit your project, do the following steps:
1. Ensure you pass all points on the [rubric](https://review.udacity.com/#!/rubrics/1004/view).
2. Submit the following in a zip file.
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.